<a href="https://www.kaggle.com/code/dascient/yahoo-s-top-gainers-kalman-forecasting-model?scriptVersionId=227569128" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Welcome to our Universe
## [@donutz.ai](www.donutz.ai/#)

- Information and market data collected from Yahoo - https://finance.yahoo.com/gainers/
- Please do not consider the recommendations below as valid. Trade at your own discretion. 

In [1]:
# source
# https://github.com/twopirllc/pandas-ta/blob/main/examples/example.ipynb

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install yfinance --upgrade --no-cache-dir
!pip install plotly
!pip install pykalman
!pip install pandas_ta

from IPython.display import clear_output
import plotly.graph_objects as go
import warnings,time,ast
from pykalman import KalmanFilter
import plotly.graph_objects as go
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from dateutil.tz import tzlocal
from rich import print, pretty
from datetime import datetime
from random import randint
from random import seed
import pandas_ta as ta
import yfinance as yf
import pandas as pd
import numpy as np
import lxml.html
import requests
import os

clear_output()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
def calculate_symbol(df):
    
    df['close'] = df['Close']
    df['low'] = df['Low']
    df['high'] = df['High']
    df['open'] = df['Open']
    
    close = df['close'][len(df)-1]
    low = df['low'][len(df)-1]

    # Construct a Kalman filter
    kf = KalmanFilter(transition_matrices = [1],    # The value for At. It is a random walk so is set to 1.0
                      observation_matrices = [1],   # The value for Ht.
                      initial_state_mean = 0,       # Any initial value. It will converge to the true state value.
                      initial_state_covariance = 1, # Sigma value for the Qt in Equation (1) the Gaussian distribution
                      observation_covariance=1,     # Sigma value for the Rt in Equation (2) the Gaussian distribution
                      transition_covariance=.01)    # A small turbulence in the random walk parameter 1.0
    # Get the Kalman smoothing
    state_means, _ = kf.filter(df['close'].values)
    # Call it kf_mean
    df['kf_mean'] = np.array(state_means)
    kalman = df.kf_mean[len(df)-1]
    aboveKalman = low > kalman


    # exponential moving averages 
    ema_13 = df.ta.ema(13, append=True)[-1:].reset_index(drop=True)[0]
    ema_31 = df.ta.ema(31, append=True)[-1:].reset_index(drop=True)[0]
    ema_crossover = ema_13 > kalman


    # lower/upper 14-day bollinger bands for mean reversion
    bbl_14 = df.ta.bbands(length=14, append=True)[['BBL_14_2.0']].tail(1).values[0][0]
    bbu_14 = df.ta.bbands(length=14, append=True)[['BBU_14_2.0']].tail(1).values[0][0]
    bband_buy = close < bbl_14
    bband_sell = close > bbu_14


    # ichimoku 9 & 26-day forecasts 
    # https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html#ta.trend.IchimokuIndicator
    isa_9 = df.ta.ichimoku()[1]['ISA_9'].tail(1).values[0] # help(ta.ichimoku)
    isb_26 = df.ta.ichimoku()[1]['ISB_26'].tail(1).values[0]


    # archer ma 
    # https://github.com/twopirllc/pandas-ta#general
    amat = (df.ta.amat()['AMATe_LR_8_21_2'].tail(1).values[0] == 1)


    # rsi
    rsi = df.ta.rsi()[len(df)-1]
    rsi_buy = rsi < 30
    rsi_sell = rsi > 70


    # choppy
    # https://github.com/twopirllc/pandas-ta#trend-18
    try: 
        chop = "{:.2f}".format(df.ta.chop()[len(df.ta.chop())-1]) 
    except RunTimeWarning:
        chop = 0


    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    # signal
    #buy = (close < isa_9) & (close < isb_26) & amat & rsi_buy & bband_buy & aboveKalman
    
    # kalman is accounted for in the ema_crossover
    buy = amat & ema_crossover# & aboveKalman

    #sell = (close > isa_9) & (close > isb_26) & ~amat & rsi_sell & bband_sell & ~aboveKalman
    
    # kalman is accounted for in the ema_crossover
    sell = ~amat & ~ema_crossover# & ~aboveKalman
    
    return df, close, isa_9, isb_26, chop, rsi, amat, ema_crossover, buy, sell, aboveKalman

# plotter
def plot(df,symbol):
    fig = go.Figure(go.Candlestick(x=df.index,
                                   open=df['open'],
                                   high=df['high'],
                                   low=df['low'],
                                   close=df['close'],
                                   name=symbol,
                                  ))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['kf_mean'], 
                             opacity=0.7, 
                             line=dict(color='purple', width=2), 
                             name='Kalman Filter'))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['EMA_13'], 
                             opacity=0.7, 
                             line=dict(color='orange', width=2), 
                             name='EMA_13'))
    
    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['EMA_31'], 
                             opacity=0.7, 
                             line=dict(color='lightblue', width=2), 
                             name='EMA_31 '))
    
    fig.update_layout(title=f'Ticker: {symbol}')
    fig.update_layout(xaxis_rangeslider_visible=False)
    return fig.show()

# [Fear & Greed Index](https://www.cnn.com/markets/fear-and-greed)

In [3]:
!pip install fear-and-greed
clear_output()

import fear_and_greed
#type(fear_and_greed.get())

print('\nIndex: ',fear_and_greed.get()[0])
print('Range: ',fear_and_greed.get()[1])
print('Datetime: ',fear_and_greed.get()[2])

Index:  15.8753968253968

Range:  extreme fear

Datetime:  2025-03-14 12:00:16+00:00

# Today's Yahoo Gainers 
## Including Recommendations, Candlesticks, Closing Price, Ichimoku 9 & 26 Day Spans, Choppiness, RSI, Exponential Moving Averages (EMA) & Kalman Forecast Predictors. 
### This notebook only plots entry & exit stocks out of the 100 from Yahoo's Gainers webpage.

In [4]:
from datetime import datetime
today = pd.Timestamp(datetime.now(), tz='EST').strftime("%Y-%m-%d")
print(f"Calculating indicators for today's date: {today}.")

Calculating indicators for today's date: 2025-03-14.

# Entry Points

In [5]:
!pip install bs4
clear_output()
from bs4 import BeautifulSoup

url = "https://finance.yahoo.com/markets/stocks/gainers/"
soup = BeautifulSoup(requests.get(url).text, 'html.parser')
assets = soup.find_all('a', attrs={"class":"Fw(600)"})

In [6]:
gainers = pd.read_html('https://www.tradingview.com/markets/stocks-usa/market-movers-large-cap/')
gainers[0]

,Symbol,Market cap,Price,Change %,Volume,Rel Volume,P/E,EPS dilTTM,EPS dil growthTTM YoY,Div yield %TTM,Sector,Analyst Rating
0,AAPLApple Inc.,3.15 T USD,209.68 USD,−3.36%,61.37 M,1.12,33.33,6.29 USD,−2.13%,0.46%,Electronic technology,Buy
1,NVDANVIDIA Corporation,2.82 T USD,115.58 USD,−0.14%,299.02 M,0.82,39.32,2.94 USD,+146.27%,0.03%,Electronic technology,Strong buy
2,MSFTMicrosoft Corporation,2.82 T USD,378.77 USD,−1.17%,20.47 M,0.78,30.51,12.42 USD,+12.29%,0.82%,Technology services,Strong buy
3,"AMZNAmazon.com, Inc.",2.05 T USD,193.89 USD,−2.51%,41.27 M,0.82,35.11,5.52 USD,+91.10%,0.00%,Retail trade,Strong buy
4,GOOGAlphabet Inc.,1.99 T USD,164.73 USD,−2.53%,15.21 M,0.64,20.47,8.05 USD,+38.60%,0.36%,Technology services,Buy
...,...,...,...,...,...,...,...,...,...,...,...,...
95,INTCIntel Corporation,102.62 B USD,23.70 USD,+14.60%,243.26 M,2.23,—,−4.37 USD,"−1,233.49%",2.42%,Electronic technology,Neutral
96,"ADIAnalog Devices, Inc.",101.08 B USD,203.80 USD,−1.36%,2.63 M,0.59,64.99,3.14 USD,−43.88%,1.78%,Electronic technology,Buy
97,"ANETArista Networks, Inc.",101.07 B USD,80.14 USD,−0.14%,6.97 M,0.55,36.00,2.23 USD,+35.34%,0.00%,Electronic technology,Buy
98,"MELIMercadoLibre, Inc.",100.87 B USD,"1,989.70 USD",−0.75%,510.69 K,1.00,52.79,37.69 USD,+93.80%,0.00%,Retail trade,Strong buy


In [7]:
from IPython.core.display import display, HTML
display(HTML("""
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <!-- Optimized for mobile & low-end devices -->
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Optimized Financial Dashboard</title>
  <!-- Load Plotly asynchronously -->
  <script src="https://cdn.plot.ly/plotly-latest.min.js" defer></script>
  <style>
    /* Minimal CSS for responsiveness and performance */
    body { font-family: Arial, sans-serif; margin: 10px; }
    h1 { text-align: center; font-size: 1.5em; }
    .tab { overflow: hidden; border: 1px solid #ccc; background-color: #f1f1f1; }
    .tab button {
      background-color: inherit;
      float: left;
      border: none;
      outline: none;
      cursor: pointer;
      padding: 10px 16px;
      transition: 0.2s;
      font-size: 1em;
    }
    .tab button:hover { background-color: #ddd; }
    .tab button.active { background-color: #ccc; }
    .tabcontent { display: none; padding: 10px 12px; border: 1px solid #ccc; border-top: none; }
  </style>
</head>
<body>
  <h1>Optimized Financial Dashboard</h1>
  
  <!-- Navigation Tabs -->
  <div class="tab">
    <button class="tablinks" onclick="openTab(event, 'Overview')" id="defaultOpen">Overview</button>
    <button class="tablinks" onclick="openTab(event, 'Histogram')">Histogram</button>
    <button class="tablinks" onclick="openTab(event, 'Radar')">Radar</button>
    <button class="tablinks" onclick="openTab(event, 'Parallel')">Parallel</button>
  </div>
  
  <!-- Tab Content -->
  <div id="Overview" class="tabcontent">
    <h3>Overview: Scatter Plot</h3>
    <div id="scatterPlot" style="width:100%; height:400px;"></div>
  </div>
  
  <div id="Histogram" class="tabcontent">
    <h3>Histogram: Price Distribution</h3>
    <div id="histogramPlot" style="width:100%; height:400px;"></div>
  </div>
  
  <div id="Radar" class="tabcontent">
    <h3>Radar Chart</h3>
    <div id="radarChart" style="width:100%; height:400px;"></div>
  </div>
  
  <div id="Parallel" class="tabcontent">
    <h3>Parallel Coordinates</h3>
    <div id="parallelPlot" style="width:100%; height:400px;"></div>
  </div>
  
  <script defer>
    // Optimized inline script for low-end devices & Kaggle Notebook environment

    // Sample dataset (kept small for performance)
    var data = [
      { "Symbol": "AAPL", "MarketCap": "3.15 T", "Price": 209.68, "Change": -3.36, "Volume": "61.37 M", "Sector": "Electronic tech", "P/E": 33.33, "EPS": 6.29, "EPSGrowth": -2.13, "DivYield": 0.46 },
      { "Symbol": "NVDA", "MarketCap": "2.82 T", "Price": 115.58, "Change": -0.14, "Volume": "299.02 M", "Sector": "Electronic tech", "P/E": 39.32, "EPS": 2.94, "EPSGrowth": 146.27, "DivYield": 0.03 },
      { "Symbol": "MSFT", "MarketCap": "2.82 T", "Price": 378.77, "Change": -1.17, "Volume": "20.47 M", "Sector": "Tech services", "P/E": 30.51, "EPS": 12.42, "EPSGrowth": 12.29, "DivYield": 0.82 },
      { "Symbol": "AMZN", "MarketCap": "2.05 T", "Price": 193.89, "Change": -2.51, "Volume": "41.27 M", "Sector": "Retail", "P/E": 35.11, "EPS": 5.52, "EPSGrowth": 91.10, "DivYield": 0.00 },
      { "Symbol": "GOOG", "MarketCap": "1.99 T", "Price": 164.73, "Change": -2.53, "Volume": "15.21 M", "Sector": "Tech services", "P/E": 20.47, "EPS": 8.05, "EPSGrowth": 38.60, "DivYield": 0.36 }
    ];
    
    // Helper: Convert magnitude strings (e.g., "3.15 T") to numbers
    function parseValue(value) {
      if (typeof value === 'string') {
        var parts = value.split(" ");
        if (parts.length >= 2) {
          var num = parseFloat(parts[0]);
          var suffix = parts[1].toUpperCase();
          if (suffix === "T") return num * 1e12;
          if (suffix === "B") return num * 1e9;
          if (suffix === "M") return num * 1e6;
          if (suffix === "K") return num * 1e3;
        }
      }
      return parseFloat(value);
    }
    
    // Preprocess data: add numeric values for MarketCap and Volume
    data.forEach(function(d) {
      d.MarketCapNum = parseValue(d.MarketCap);
      d.VolumeNum = parseValue(d.Volume);
    });
    
    // Draw Overview Scatter Plot
    function drawScatter() {
      var trace = {
        x: data.map(d => d.MarketCapNum),
        y: data.map(d => d.Price),
        mode: 'markers',
        type: 'scatter',
        text: data.map(d => d.Symbol),
        marker: {
          size: data.map(d => Math.sqrt(d.VolumeNum) / 500),
          color: data.map(d => d.Change),
          colorscale: 'RdYlGn',
          colorbar: { title: 'Change %' }
        }
      };
      var layout = {
        title: 'Price vs Market Cap',
        xaxis: { title: 'Market Cap (numeric)' },
        yaxis: { title: 'Price (USD)' },
        margin: { l: 50, r: 50, t: 50, b: 50 }
      };
      Plotly.newPlot('scatterPlot', [trace], layout, {responsive: true});
    }
    
    // Draw Histogram of Price Distribution
    function drawHistogram() {
      var trace = {
        x: data.map(d => d.Price),
        type: 'histogram',
        nbinsx: 15
      };
      var layout = {
        title: 'Price Distribution',
        xaxis: { title: 'Price (USD)' },
        yaxis: { title: 'Count' },
        margin: { l: 50, r: 50, t: 50, b: 50 }
      };
      Plotly.newPlot('histogramPlot', [trace], layout, {responsive: true});
    }
    
    // Draw Radar Chart (using first stock as example)
    function drawRadar() {
      var d = data[0];
      var categories = ['P/E', 'EPS', 'EPSGrowth', 'DivYield', 'Price'];
      var values = [d["P/E"], d.EPS, d.EPSGrowth, d.DivYield, d.Price];
      // Close the loop for radar chart
      categories.push(categories[0]);
      values.push(values[0]);
      
      var trace = {
        type: 'scatterpolar',
        r: values,
        theta: categories,
        fill: 'toself',
        name: d.Symbol
      };
      var layout = {
        polar: { radialaxis: { visible: true } },
        title: 'Radar Chart: ' + d.Symbol,
        margin: { l: 50, r: 50, t: 50, b: 50 }
      };
      Plotly.newPlot('radarChart', [trace], layout, {responsive: true});
    }
    
    // Draw Parallel Coordinates Plot
    function drawParallel() {
      var dimensions = [
        { label: 'Price', values: data.map(d => d.Price) },
        { label: 'Market Cap', values: data.map(d => d.MarketCapNum) },
        { label: 'Volume', values: data.map(d => d.VolumeNum) },
        { label: 'P/E', values: data.map(d => d["P/E"] || 0) },
        { label: 'EPS', values: data.map(d => d.EPS) },
        { label: 'EPSGrowth', values: data.map(d => d.EPSGrowth) },
        { label: 'DivYield', values: data.map(d => d.DivYield) }
      ];
      var trace = {
        type: 'parcoords',
        dimensions: dimensions,
        line: {
          color: data.map(d => d.Price),
          colorscale: 'Viridis'
        }
      };
      var layout = {
        title: 'Parallel Coordinates',
        margin: { l: 50, r: 50, t: 50, b: 50 }
      };
      Plotly.newPlot('parallelPlot', [trace], layout, {responsive: true});
    }
    
    // Tab Switching Logic
    function openTab(evt, tabName) {
      var i, tabcontent, tablinks;
      tabcontent = document.getElementsByClassName("tabcontent");
      for (i = 0; i < tabcontent.length; i++) {
        tabcontent[i].style.display = "none";
      }
      tablinks = document.getElementsByClassName("tablinks");
      for (i = 0; i < tablinks.length; i++) {
        tablinks[i].className = tablinks[i].className.replace(" active", "");
      }
      document.getElementById(tabName).style.display = "block";
      evt.currentTarget.className += " active";
    }
    
    // Initialize: wait for DOM content to load
    document.addEventListener("DOMContentLoaded", function() {
      document.getElementById("defaultOpen").click();
      drawScatter();
      drawHistogram();
      drawRadar();
      drawParallel();
    });
  </script>
</body>
</html>
"""))


In [8]:
# Required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
%matplotlib inline

# ------------------------------
# 1. Download Data
# ------------------------------
# Download full OHLCV data (3 months, daily interval) for a couple of tickers.
tickers = ['ESPGY','ALYI']
data = yf.download(tickers=tickers, period='3mo', interval='1d')

# ------------------------------
# 2. Define Technical Indicator Function
# ------------------------------
def calculate_symbol(df):
    """
    Dummy calculation of technical indicators from a DataFrame (df) containing
    at least 'Open', 'High', 'Low', 'Close' columns.
    
    Returns a tuple with:
      index 1: last close price
      index 2: 9-day moving average (as a proxy for Ichimoku 9-day forecast)
      index 3: 26-day moving average (as a proxy for Ichimoku 26-day forecast)
      index 4: 'Choppiness' (dummy: standard deviation of close prices)
      index 5: RSI (dummy: fixed value 50)
      index 6: Archer MA Trending (dummy: True)
      index 7: EMA_13 > Kalman (dummy boolean, computed below)
      index 8: Entry Condition (True if EMA_13 > Kalman)
      index 9: Exit Condition (complement of above)
      index 10: Low > Kalman (dummy boolean)
    """
    if df.shape[0] < 30:
        raise ValueError("Not enough data for calculation.")
    
    close = df['Close']
    last_close = close.iloc[-1]
    ichimoku9 = close.rolling(window=9).mean().iloc[-1]
    ichimoku26 = close.rolling(window=26).mean().iloc[-1] if len(close) >= 26 else np.nan
    choppiness = close.std()  # Dummy: using standard deviation as a proxy
    rsi = 50  # Dummy fixed value for RSI
    archer_ma_trending = True  # Dummy value
    
    # Calculate a dummy 13-day EMA
    ema_13 = close.ewm(span=13, adjust=False).mean().iloc[-1]
    # Dummy "Kalman" value: slightly lower than the EMA
    kalman = ema_13 * 0.99
    
    # Entry condition: For example, if EMA_13 > Kalman (always True in this dummy)
    entry_condition = (ema_13 > kalman)
    exit_condition = not entry_condition
    # Dummy condition: last Low > kalman value
    low_gt_kalman = (df['Low'].iloc[-1] > kalman)
    
    return (None, last_close, ichimoku9, ichimoku26, choppiness, rsi, 
            archer_ma_trending, ema_13 > kalman, entry_condition, exit_condition, low_gt_kalman)

# ------------------------------
# 3. Define a Plotting Function
# ------------------------------
def plot(df, ticker):
    """
    Plot the last 90 days of Close prices for a given ticker using Plotly.
    """
    # Use only the tail(90) rows for plotting
    fig = px.line(df.tail(90), x=df.tail(90).index, y='Close',
                  title=f"{ticker} - Last 90 Days Close Price")
    fig.update_layout(xaxis_title="Date", yaxis_title="Close Price (USD)")
    fig.show()

# ------------------------------
# 4. Process Each Ticker and Display Results
# ------------------------------
print('\n Entry Points:')

# When multiple tickers are downloaded, data comes as a DataFrame with MultiIndex columns.
# We loop through each ticker's data.
if isinstance(data.columns, pd.MultiIndex):
    tickers_list = data.columns.levels[1]
else:
    tickers_list = tickers

for ticker in tickers_list:
    try:
        # For multi-ticker data, extract the ticker-specific DataFrame.
        if isinstance(data.columns, pd.MultiIndex):
            df_ticker = data.xs(ticker, axis=1, level=1)
        else:
            df_ticker = data
        
        # Calculate technical indicators for this ticker.
        result = calculate_symbol(df_ticker)
        
        # Check entry condition: entry_condition is at index 8, and dummy choppiness (index 4) must be <= 40.
        if result[8] == True and result[4] <= 40:
            print(f'\n\nTicker: {ticker}')
            print(' "Entry Point"  (EMA_13 > Kalman & Choppiness <= 40):', result[8])
            print(' "Exit Point"  (Opposite of above):', result[9])
            print('\n Close Price:', result[1])
            print(' Ichimoku 9-Day Forecast (9-day MA):', result[2])
            print(' Ichimoku 26-Day Forecast (26-day MA):', result[3])
            print(' Choppiness (std. dev.):', result[4])
            print(' RSI (dummy):', result[5])
            print(' Archer MA Trending (dummy):', result[6])
            print(' EMA_13 > Kalman:', result[7])
            print(' Low > Kalman:', result[10])
            # Plot the ticker's last 90 days of Close prices.
            plot(df_ticker, ticker)
        else:
            print(f"{ticker}: Conditions not met.")
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed


Entry Points:

Ticker: ALYI

"Entry Point"  (EMA_13 > Kalman & Choppiness <= 40): True

...

Choppiness (std. dev.): 0.004774273440310532

RSI (dummy): 50

Archer MA Trending (dummy): True

EMA_13 > Kalman: True

Low > Kalman: True

# Portfolio

In [9]:
from IPython.core.display import display, HTML
display(HTML("""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <!-- Ensure mobile and low‑end device compatibility -->
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>ArchCore</title>
  <!-- Futuristic Neon Font -->
  <link href="https://fonts.googleapis.com/css2?family=Orbitron&display=swap" rel="stylesheet">
  <style>
    /* CSS Variables for theming */
    :root {
      --primary-color: #2575fc;
      --secondary-color: #6a11cb;
      --accent-color: #ff00ff;
      --neon-cyan: #00ffff;
      --bg-dark: #1e1e1e;
      --text-color: #fff;
      --spinner-size: 40px;
    }
    /* Global Reset & Font */
    * { box-sizing: border-box; margin: 0; padding: 0; }
    body {
      font-family: 'Orbitron', sans-serif;
      color: var(--text-color);
      line-height: 1.6;
      background: linear-gradient(135deg, var(--secondary-color), var(--primary-color), var(--secondary-color));
      background-size: 400% 400%;
      animation: gradientBG 15s ease infinite;
      padding-top: 100px; /* for header */
    }
    @-webkit-keyframes gradientBG {
      0% { background-position: 0% 50%; }
      50% { background-position: 100% 50%; }
      100% { background-position: 0% 50%; }
    }
    @keyframes gradientBG {
      0% { background-position: 0% 50%; }
      50% { background-position: 100% 50%; }
      100% { background-position: 0% 50%; }
    }
    /* Header */
    header {
      position: fixed;
      top: 0; left: 0;
      width: 100%;
      background-color: rgba(30,30,30,0.95);
      padding: 15px 20px;
      z-index: 120;
      display: flex;
      justify-content: space-between;
      align-items: center;
      border-bottom: 3px solid rgba(255,255,255,0.2);
    }
    header h1 {
      font-size: 2.8em;
      color: #ffffff;
      text-shadow: 0 0 10px var(--accent-color), 0 0 20px var(--neon-cyan);
      animation: pulseGlow 2s infinite alternate;
    }
    @-webkit-keyframes pulseGlow {
      from { text-shadow: 0 0 10px var(--accent-color), 0 0 20px var(--neon-cyan); }
      to { text-shadow: 0 0 20px var(--accent-color), 0 0 30px var(--neon-cyan); }
    }
    @keyframes pulseGlow {
      from { text-shadow: 0 0 10px var(--accent-color), 0 0 20px var(--neon-cyan); }
      to { text-shadow: 0 0 20px var(--accent-color), 0 0 30px var(--neon-cyan); }
    }
    .header-controls {
      display: flex;
      align-items: center;
      gap: 15px;
    }
    .header-controls button {
      padding: 8px 16px;
      font-size: 1em;
      border: 2px solid var(--accent-color);
      border-radius: 6px;
      background-color: rgba(0,0,0,0.35);
      color: var(--text-color);
      cursor: pointer;
      transition: all 0.3s ease;
    }
    .header-controls button:hover {
      background-color: rgba(0,0,0,0.55);
      box-shadow: 0 0 12px var(--accent-color), 0 0 18px var(--neon-cyan);
    }
    /* Inline Marquee (placed in middle section) */
    .marquee-inline {
      width: 100%;
      background: rgba(0,0,0,0.85);
      padding: 5px 0;
      margin: 20px 0;
      text-align: center;
    }
    .marquee-inline .tradingview-widget-container { width: 100% !important; }
    /* Main Container */
    .container { max-width: 1600px; margin: 0 auto; padding: 20px; }
    /* Dashboard Grid Layout */
    .dashboard-grid {
      display: grid;
      grid-template-columns: 320px auto;
      gap: 40px;
      margin-bottom: 40px;
    }
    /* Sidebar */
    aside.sidebar {
      background-color: rgba(30,30,30,0.97);
      padding: 25px;
      border-radius: 10px;
      border: 2px solid rgba(255,255,255,0.25);
      box-shadow: 0 0 20px var(--accent-color), 0 0 30px var(--neon-cyan);
      position: sticky;
      top: 120px;
    }
    aside.sidebar h2 {
      font-size: 1.8em;
      margin-bottom: 20px;
      border-bottom: 2px solid rgba(255,255,255,0.2);
      padding-bottom: 10px;
      text-shadow: 0 0 6px var(--accent-color), 0 0 12px var(--neon-cyan);
    }
    /* Main Content Area */
    main.main-content { display: flex; flex-direction: column; gap: 40px; }
    /* Live Groups Container */
    .live-groups-container { display: grid; grid-template-columns: repeat(2, 1fr); gap: 40px; }
    .group {
      background-color: rgba(30,30,30,0.95);
      border: 2px solid rgba(255,255,255,0.25);
      border-radius: 10px;
      box-shadow: 0 0 20px var(--accent-color), 0 0 30px var(--neon-cyan);
      padding: 15px;
      display: flex; flex-direction: column; gap: 20px;
      position: relative;
    }
    .group h3 {
      font-size: 1.6em;
      border-bottom: 2px solid rgba(255,255,255,0.2);
      padding-bottom: 8px;
      text-shadow: 0 0 5px var(--accent-color), 0 0 10px var(--neon-cyan);
    }
    .group .group-controls {
      display: flex; align-items: center; gap: 10px; margin-bottom: 10px;
    }
    .group .group-controls label { font-size: 1em; }
    .group .group-controls input[type="text"] {
      padding: 8px; font-size: 1em; border: 2px solid var(--neon-cyan);
      border-radius: 4px; width: 120px; background: rgba(0,0,0,0.25); color: var(--text-color);
      transition: all 0.3s ease;
    }
    .group .group-controls button {
      padding: 8px 16px; font-size: 1em; border: 2px solid var(--accent-color);
      border-radius: 4px; background-color: rgba(0,0,0,0.35); color: var(--text-color);
      cursor: pointer; transition: all 0.3s ease;
    }
    .group .group-controls button:hover {
      background-color: rgba(0,0,0,0.55);
      box-shadow: 0 0 12px var(--accent-color), 0 0 18px var(--neon-cyan);
    }
    .group .chart, .group .analysis, .group .symbol {
      border: 2px solid rgba(255,255,255,0.3);
      border-radius: 8px;
      box-shadow: 0 0 15px var(--neon-cyan);
      overflow: hidden;
    }
    .group .chart { height: 280px; }
    .group .analysis, .group .symbol { height: 240px; }
    /* Panels */
    .panel {
      background-color: rgba(30,30,30,0.95);
      border: 2px solid rgba(255,255,255,0.25);
      border-radius: 10px;
      padding: 20px;
      position: relative;
      box-shadow: 0 0 20px var(--accent-color), 0 0 30px var(--neon-cyan);
    }
    .panel h2 {
      font-size: 1.8em; margin-bottom: 10px;
      text-shadow: 0 0 5px var(--accent-color), 0 0 10px var(--neon-cyan);
    }
    /* Last Updated: very small and flush upper right */
    .last-updated {
      position: absolute; top: 5px; right: 5px;
      color: #ccc; font-size: 0.4em;
    }
    /* Tables */
    table {
      width: 100%; border-collapse: collapse;
      background: rgba(0,0,0,0.45); border-radius: 6px; overflow: hidden;
    }
    table th, table td {
      border: 1px solid rgba(255,255,255,0.3);
      padding: 8px; text-align: center; word-break: break-word;
    }
    table th { background: rgba(0,0,0,0.65); }
    /* Additional Analytics Section */
    .additional-analytics { max-width: 1600px; margin: 40px auto; padding: 20px; display: grid; grid-template-columns: 1fr; gap: 40px; }
    .analytics-grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 40px; }
    /* Modal for Help/Settings */
    .modal {
      position: fixed; top: 50%; left: 50%;
      transform: translate(-50%, -50%);
      background-color: var(--bg-dark);
      border: 2px solid var(--accent-color);
      border-radius: 10px; padding: 20px;
      width: 90%; max-width: 600px;
      z-index: 130; display: none;
      box-shadow: 0 0 30px var(--neon-cyan);
    }
    .modal h2 { margin-bottom: 15px; }
    .modal p { margin-bottom: 10px; line-height: 1.4; }
    .modal-close {
      position: absolute; top: 10px; right: 10px;
      background: none; border: none; font-size: 1.5em;
      color: var(--accent-color); cursor: pointer;
    }
    /* Responsive Design */
    @media (max-width: 768px) {
      .dashboard-grid { grid-template-columns: 1fr; }
      aside.sidebar { position: relative; top: 0; }
      .live-groups-container { grid-template-columns: 1fr; }
      .analytics-grid { grid-template-columns: 1fr; }
      header h1 { font-size: 2em; }
    }
  </style>
</head>
<body>
  <!-- Header -->
  <header>
    <h1>ArchCore</h1>
    <div class="header-controls">
      <button id="themeToggle" title="Toggle Dark/Light Mode">Toggle Theme</button>
      <button id="helpToggle" title="Show/Hide Instructions">Help</button>
    </div>
  </header>
  
  <!-- Inline Marquee in Middle -->
  <div class="container">
    <div class="marquee-inline">
      <div class="tradingview-widget-container">
        <div class="tradingview-widget-container__widget"></div>
        <script type="text/javascript" src="https://s3.tradingview.com/external-embedding/embed-widget-ticker-tape.js" async>
        {
          "symbols": [
            {"proName": "FOREXCOM:SPXUSD", "title": "S&P 500"},
            {"proName": "FOREXCOM:NSXUSD", "title": "Nasdaq"},
            {"proName": "FX_IDC:EURUSD", "title": "EUR/USD"},
            {"proName": "FOREXCOM:DJI", "title": "Dow Jones"},
            {"proName": "NASDAQ:TSLA", "title": "Tesla"},
            {"proName": "NASDAQ:AAPL", "title": "Apple"},
            {"proName": "NASDAQ:AMZN", "title": "Amazon"},
            {"proName": "NASDAQ:GOOGL", "title": "Google"},
            {"proName": "NYSE:IBM", "title": "IBM"},
            {"proName": "NYSE:GE", "title": "GE"}
          ],
          "colorTheme": "dark",
          "isTransparent": false,
          "displayMode": "regular",
          "locale": "en"
        }
        </script>
      </div>
    </div>
  </div>
  
  <!-- Main Dashboard -->
  <div class="container">
    <div class="dashboard-grid">
      <!-- Sidebar: Short Squeeze Leaderboard -->
      <aside class="sidebar">
        <h2>Short Squeeze Leaderboard</h2>
        <div id="shortSqueezeSidebar">
          <div class="spinner"></div>
        </div>
      </aside>
      <!-- Main Content: Live Groups -->
      <main class="main-content">
        <div class="live-groups-container">
          <!-- Group 1 -->
          <div class="group">
            <h3>Group 1</h3>
            <div class="group-controls">
              <label for="group1TickerInput" title="Enter ticker for Group 1">Ticker:</label>
              <input type="text" id="group1TickerInput" value="TSLA">
              <button onclick="updateGroupTicker(1)" title="Update Group 1">Update</button>
            </div>
            <div class="chart" id="live_chart_1"></div>
            <div class="analysis" id="ta_widget_1"></div>
            <div class="symbol" id="symbol_info_1"></div>
          </div>
          <!-- Group 2 -->
          <div class="group">
            <h3>Group 2 (Gold Stocks)</h3>
            <div class="group-controls">
              <label for="group2TickerInput" title="Enter ticker for Group 2">Ticker:</label>
              <input type="text" id="group2TickerInput" value="NEM">
              <button onclick="updateGroupTicker(2)" title="Update Group 2">Update</button>
            </div>
            <div class="chart" id="live_chart_2"></div>
            <div class="analysis" id="ta_widget_2"></div>
            <div class="symbol" id="symbol_info_2"></div>
          </div>
          <!-- Group 3 -->
          <div class="group">
            <h3>Group 3 (High Dividend)</h3>
            <div class="group-controls">
              <label for="group3TickerInput" title="Enter ticker for Group 3">Ticker:</label>
              <input type="text" id="group3TickerInput" value="T">
              <button onclick="updateGroupTicker(3)" title="Update Group 3">Update</button>
            </div>
            <div class="chart" id="live_chart_3"></div>
            <div class="analysis" id="ta_widget_3"></div>
            <div class="symbol" id="symbol_info_3"></div>
          </div>
          <!-- Group 4 -->
          <div class="group">
            <h3>Group 4 (Top Gainers)</h3>
            <div class="group-controls">
              <label for="group4TickerInput" title="Enter ticker for Group 4">Ticker:</label>
              <input type="text" id="group4TickerInput" value="AMD">
              <button onclick="updateGroupTicker(4)" title="Update Group 4">Update</button>
            </div>
            <div class="chart" id="live_chart_4"></div>
            <div class="analysis" id="ta_widget_4"></div>
            <div class="symbol" id="symbol_info_4"></div>
          </div>
        </div>
      </main>
    </div>
  </div>
  
  <!-- Additional Analytics Section -->
  <div class="container additional-analytics">
    <!-- Row 1: Volume Spike & News Sentiment -->
    <div class="analytics-grid">
      <div class="panel">
        <h2>Volume Spike Leaderboard <span class="last-updated" id="volumeUpdated">Last Updated: --</span></h2>
        <div id="volumeSpikeLeaderboard"><div class="spinner"></div></div>
      </div>
      <div class="panel">
        <h2>News Sentiment Analysis <span class="last-updated" id="sentimentUpdated">Last Updated: --</span></h2>
        <div id="newsSentiment"><div class="spinner"></div></div>
      </div>
    </div>
    <!-- Row 2: Breakout Alerts & Earnings Calendar -->
    <div class="analytics-grid">
      <div class="panel">
        <h2>Breakout Alerts <span class="last-updated" id="breakoutUpdated">Last Updated: --</span></h2>
        <div id="breakoutAlerts"><div class="spinner"></div></div>
      </div>
      <div class="panel">
        <h2>Earnings Calendar <span class="last-updated" id="earningsUpdated">Last Updated: --</span></h2>
        <div id="earningsCalendar">
          <table>
            <thead>
              <tr><th>Company</th><th>Ticker</th><th>Earnings Date</th></tr>
            </thead>
            <tbody>
              <tr><td>Apple Inc.</td><td>AAPL</td><td>04/27/2025</td></tr>
              <tr><td>Amazon.com</td><td>AMZN</td><td>04/28/2025</td></tr>
              <tr><td>Alphabet Inc.</td><td>GOOGL</td><td>04/29/2025</td></tr>
            </tbody>
          </table>
          <p style="text-align:center; font-size:0.9em; color:#ccc;">* Replace with live earnings API data *</p>
        </div>
      </div>
    </div>
    <!-- Row 3: Insider Trading Activity -->
    <div class="analytics-grid">
      <div class="panel" style="grid-column: 1 / -1;">
        <h2>Insider Trading Activity <span class="last-updated" id="insiderUpdated">Last Updated: --</span></h2>
        <div id="insiderTrading">
          <table>
            <thead>
              <tr><th>Company</th><th>Ticker</th><th>Insider Buy/Sell</th><th>Volume</th></tr>
            </thead>
            <tbody>
              <tr><td>Microsoft</td><td>MSFT</td><td>Buy</td><td>10,000</td></tr>
              <tr><td>Tesla Inc.</td><td>TSLA</td><td>Sell</td><td>5,000</td></tr>
            </tbody>
          </table>
          <p style="text-align:center; font-size:0.9em; color:#ccc;">* Make your decisions under grace and with a sound mind. *</p>
        </div>
      </div>
    </div>
    <!-- Row 4: Volatility Monitor & Moving Average Crossover Alerts -->
    <div class="analytics-grid">
      <div class="panel">
        <h2>Volatility Monitor <span class="last-updated" id="volatilityUpdated">Last Updated: --</span></h2>
        <div id="volatilityMonitor"><div class="spinner"></div></div>
      </div>
      <div class="panel">
        <h2>Moving Average Crossover Alerts <span class="last-updated" id="crossoverUpdated">Last Updated: --</span></h2>
        <div id="crossoverAlerts"><div class="spinner"></div></div>
      </div>
    </div>
    <!-- Row 5: Price Alert Monitor -->
    <div class="analytics-grid">
      <div class="panel">
        <h2>Price Alert Monitor</h2>
        <div id="priceAlertPanel">
          <div style="display:flex; gap:10px; flex-wrap:wrap; margin-bottom:10px;">
            <input type="text" id="alertTicker" placeholder="Ticker" title="Enter ticker for alert" style="width:100px;">
            <input type="number" id="alertPrice" placeholder="Alert Price" title="Enter your target price" style="width:100px;">
            <button onclick="checkPriceAlert()" title="Check Price">Check Price</button>
          </div>
          <div id="priceAlertResult"></div>
        </div>
      </div>
    </div>
  </div>
  
  <!-- TradingView Base Widget Script -->
  <script type="text/javascript" src="https://s3.tradingview.com/tv.js"></script>
  
  <!-- Dashboard & Enhanced Script -->
  <script>
    // Wrap initialization code to ensure DOM is loaded
    document.addEventListener("DOMContentLoaded", function() {
      // Utility: Update timestamp text in a panel
      function updateTimestamp(elementId) {
        const now = new Date();
        document.getElementById(elementId).textContent = "Last Updated: " + now.toLocaleTimeString();
      }
      
      // Theme Toggle
      const themeToggle = document.getElementById("themeToggle");
      themeToggle.addEventListener("click", () => {
        document.body.style.filter = document.body.style.filter === "invert(1)" ? "" : "invert(1)";
      });
      
      // Modal for Help/Settings (if implemented)
      const helpToggle = document.getElementById("helpToggle");
      // (Optional: Create and show a help modal here)
      
      // Live Group Widgets
      function loadLiveChartForGroup(ticker, groupId) {
        const containerId = "live_chart_" + groupId;
        document.getElementById(containerId).innerHTML = "";
        new TradingView.widget({
          "width": "100%",
          "height": 280,
          "symbol": ticker,
          "interval": "D",
          "timezone": "Etc/UTC",
          "theme": "dark",
          "style": "1",
          "locale": "en",
          "toolbar_bg": "#1e1e1e",
          "enable_publishing": false,
          "allow_symbol_change": false,
          "container_id": containerId
        });
      }
      function loadTechnicalAnalysisForGroup(ticker, groupId) {
        const containerId = "ta_widget_" + groupId;
        document.getElementById(containerId).innerHTML = "";
        const script = document.createElement('script');
        script.type = 'text/javascript';
        script.src = "https://s3.tradingview.com/external-embedding/embed-widget-technical-analysis.js";
        script.async = true;
        const config = {
          "interval": "1D",
          "width": "100%",
          "isTransparent": false,
          "height": 240,
          "symbol": ticker,
          "locale": "en"
        };
        script.innerHTML = JSON.stringify(config);
        document.getElementById(containerId).appendChild(script);
      }
      function loadSymbolInfoForGroup(ticker, groupId) {
        const containerId = "symbol_info_" + groupId;
        document.getElementById(containerId).innerHTML = "";
        const div = document.createElement('div');
        div.className = "tradingview-widget-container__widget";
        document.getElementById(containerId).appendChild(div);
        const script = document.createElement('script');
        script.type = 'text/javascript';
        script.src = "https://s3.tradingview.com/external-embedding/embed-widget-symbol-info.js";
        script.async = true;
        const config = {
          "symbol": ticker,
          "width": "100%",
          "height": 240,
          "colorTheme": "dark",
          "isTransparent": false,
          "locale": "en"
        };
        script.innerHTML = JSON.stringify(config);
        document.getElementById(containerId).appendChild(script);
      }
      function updateGroupTicker(groupId) {
        const ticker = document.getElementById("group" + groupId + "TickerInput").value.trim();
        if (!ticker) { alert("Please enter a ticker for Group " + groupId); return; }
        loadLiveChartForGroup(ticker, groupId);
        loadTechnicalAnalysisForGroup(ticker, groupId);
        loadSymbolInfoForGroup(ticker, groupId);
      }
      
      // Auto-cycle for Groups 2 & 4 (example tickers)
      let goldTickerIndex = 0;
      const goldTickers = ["NEM", "GDX", "AEM", "AUY"];
      setInterval(() => {
        goldTickerIndex = (goldTickerIndex + 1) % goldTickers.length;
        document.getElementById("group2TickerInput").value = goldTickers[goldTickerIndex];
        updateGroupTicker(2);
      }, 120000);
      
      let topGainersIndex = 0;
      const topGainers = ["AMD", "NVDA", "SQ", "ZM"];
      setInterval(() => {
        topGainersIndex = (topGainersIndex + 1) % topGainers.length;
        document.getElementById("group4TickerInput").value = topGainers[topGainersIndex];
        updateGroupTicker(4);
      }, 120000);
      
      // Short Squeeze Leaderboard (using Alpha Vantage API as example)
      const stockList = ["TSLA", "AAPL", "MSFT", "AMZN", "GOOGL", "NVDA", "META", "NFLX", "BABA", "JPM"];
      function fetchShortSqueezeData(ticker) {
        const apiKey = "0M17XRJD0VADHB6I";
        const url = `https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=${ticker}&apikey=${apiKey}`;
        return fetch(url)
          .then(response => response.json())
          .then(data => {
            const ts = data["Time Series (Daily)"];
            if (!ts) return null;
            const dates = Object.keys(ts).sort();
            const prices = dates.map(date => parseFloat(ts[date]["4. close"]));
            const dailyReturns = [];
            const priceChanges = [];
            for (let i = 1; i < prices.length; i++) {
              const change = prices[i] - prices[i-1];
              priceChanges.push(change);
              dailyReturns.push(change / prices[i-1]);
            }
            const ma20 = [];
            for (let i = 19; i < prices.length; i++) {
              let sum = 0;
              for (let j = i - 19; j <= i; j++) { sum += prices[j]; }
              ma20.push(sum / 20);
            }
            const latestPrice = prices[prices.length - 1];
            const latestReturn = dailyReturns[dailyReturns.length - 1];
            const latestMA20 = ma20[ma20.length - 1];
            const premium = (latestPrice - latestMA20) / latestMA20;
            const isCandidate = (latestReturn > 0.05) && (premium > 0.03);
            let rsi = null;
            if (priceChanges.length >= 14) {
              const recent = priceChanges.slice(-14);
              let gains = 0, losses = 0;
              recent.forEach(c => { c > 0 ? gains += c : losses += Math.abs(c); });
              const avgGain = gains / 14, avgLoss = losses / 14;
              rsi = (avgLoss === 0) ? 100 : 100 - (100 / (1 + avgGain/avgLoss));
            }
            return { ticker, latestPrice, latestReturn, premium, rsi, isCandidate };
          })
          .catch(err => { console.error("Error fetching short squeeze data for", ticker, err); return null; });
      }
      function updateShortSqueezeLeaderboard() {
        const container = document.getElementById("shortSqueezeSidebar");
        container.innerHTML = "<div class='spinner'></div>";
        Promise.all(stockList.map(ticker => fetchShortSqueezeData(ticker)))
          .then(results => {
            const candidates = results.filter(r => r && r.isCandidate);
            if (candidates.length === 0) {
              container.innerHTML = "<p>No short squeeze candidates detected at this time.</p>";
              return;
            }
            candidates.sort((a, b) => b.latestReturn - a.latestReturn);
            let html = "<table><thead><tr><th>Ticker</th><th>Price ($)</th><th>Return (%)</th><th>Premium (%)</th><th>RSI</th></tr></thead><tbody>";
            candidates.forEach(c => {
              html += `<tr>
                        <td>${c.ticker}</td>
                        <td>${c.latestPrice.toFixed(2)}</td>
                        <td>${(c.latestReturn * 100).toFixed(2)}</td>
                        <td>${(c.premium * 100).toFixed(2)}</td>
                        <td>${c.rsi !== null ? c.rsi.toFixed(2) : "N/A"}</td>
                      </tr>`;
            });
            html += "</tbody></table>";
            container.innerHTML = html;
          })
          .catch(err => { container.innerHTML = `<p>Error updating leaderboard: ${err}</p>`; });
      }
      
      // Volume Spike Leaderboard
      function fetchVolumeSpikeData(ticker) {
        const apiKey = "0M17XRJD0VADHB6I";
        const url = `https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=${ticker}&apikey=${apiKey}`;
        return fetch(url)
          .then(response => response.json())
          .then(data => {
            const ts = data["Time Series (Daily)"];
            if (!ts) return null;
            const dates = Object.keys(ts).sort();
            const volumes = dates.map(date => parseInt(ts[date]["5. volume"]));
            const avgVolume = volumes.slice(-20).reduce((a, b) => a + b, 0) / 20;
            const latestVolume = volumes[volumes.length - 1];
            const spikeRatio = latestVolume / avgVolume;
            const isSpike = spikeRatio > 2;
            return { ticker, latestVolume, avgVolume, spikeRatio, isSpike };
          })
          .catch(err => { console.error("Error fetching volume data for", ticker, err); return null; });
      }
      function updateVolumeSpikeLeaderboard() {
        const container = document.getElementById("volumeSpikeLeaderboard");
        container.innerHTML = "<div class='spinner'></div>";
        Promise.all(stockList.map(ticker => fetchVolumeSpikeData(ticker)))
          .then(results => {
            const spikes = results.filter(r => r && r.isSpike);
            if (spikes.length === 0) {
              container.innerHTML = "<p>No volume spikes detected.</p>";
              updateTimestamp("volumeUpdated");
              return;
            }
            spikes.sort((a, b) => b.spikeRatio - a.spikeRatio);
            let html = "<table><thead><tr><th>Ticker</th><th>Latest Volume</th><th>Avg Volume</th><th>Spike Ratio</th></tr></thead><tbody>";
            spikes.forEach(s => {
              html += `<tr>
                        <td>${s.ticker}</td>
                        <td>${s.latestVolume}</td>
                        <td>${Math.round(s.avgVolume)}</td>
                        <td>${s.spikeRatio.toFixed(2)}</td>
                      </tr>`;
            });
            html += "</tbody></table>";
            container.innerHTML = html;
            updateTimestamp("volumeUpdated");
          })
          .catch(err => { container.innerHTML = `<p>Error updating volume leaderboard: ${err}</p>`; });
      }
      
      // News Sentiment Analysis
      const sentimentTickerList = ["AAPL", "TSLA", "AMZN", "GOOGL", "MSFT", "NFLX", "NVDA", "META", "BABA", "IBM", "ORCL", "INTC", "CSCO", "ADBE", "CRM"];
      function fetchNewsSentimentData(ticker) {
        const token = "curdh1hr01qgoble6ua0curdh1hr01qgoble6uag";
        const url = `https://finnhub.io/api/v1/news-sentiment?symbol=${ticker}&token=${token}`;
        return fetch(url)
          .then(response => response.json())
          .then(data => {
            if (!data) return null;
            return { ticker, bullish: data.bullishPercent, bearish: data.bearishPercent, companyScore: data.companyNewsScore };
          })
          .catch(err => { console.error("Error fetching sentiment for", ticker, err); return null; });
      }
      function updateNewsSentiment() {
        const container = document.getElementById("newsSentiment");
        container.style.opacity = 0;
        container.innerHTML = "<div class='spinner'></div>";
        Promise.all(sentimentTickerList.map(ticker => fetchNewsSentimentData(ticker)))
          .then(results => {
            const valid = results.filter(r => r !== null);
            let html = "<table><thead><tr><th>Ticker</th><th>Bullish (%)</th><th>Bearish (%)</th><th>News Score</th></tr></thead><tbody>";
            valid.forEach(item => {
              html += `<tr>
                        <td>${item.ticker}</td>
                        <td>${(item.bullish * 100).toFixed(1)}</td>
                        <td>${(item.bearish * 100).toFixed(1)}</td>
                        <td>${item.companyScore !== undefined ? item.companyScore.toFixed(2) : "N/A"}</td>
                      </tr>`;
            });
            html += "</tbody></table>";
            setTimeout(() => {
              container.innerHTML = html;
              container.style.opacity = 1;
              updateTimestamp("sentimentUpdated");
            }, 500);
          })
          .catch(err => { container.innerHTML = `<p>Error updating sentiment: ${err}</p>`; container.style.opacity = 1; });
      }
      
      // Breakout Alerts
      const breakoutTickerList = ["AAPL", "TSLA", "AMZN", "GOOGL", "MSFT"];
      function fetchBreakoutData(ticker) {
        const apiKey = "0M17XRJD0VADHB6I";
        const url = `https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=${ticker}&apikey=${apiKey}`;
        return fetch(url)
          .then(response => response.json())
          .then(data => {
            const ts = data["Time Series (Daily)"];
            if (!ts) return null;
            const dates = Object.keys(ts).sort();
            const prices = dates.map(date => parseFloat(ts[date]["4. close"]));
            const recentPrices = prices.slice(-20);
            const recentHigh = Math.max(...recentPrices);
            const latestPrice = prices[prices.length - 1];
            const isBreakout = latestPrice > recentHigh;
            return { ticker, latestPrice, recentHigh, isBreakout };
          })
          .catch(err => { console.error("Error fetching breakout data for", ticker, err); return null; });
      }
      function updateBreakoutAlerts() {
        const container = document.getElementById("breakoutAlerts");
        container.innerHTML = "<div class='spinner'></div>";
        Promise.all(breakoutTickerList.map(ticker => fetchBreakoutData(ticker)))
          .then(results => {
            const breakouts = results.filter(r => r && r.isBreakout);
            if (breakouts.length === 0) {
              container.innerHTML = "<p>No breakout alerts at this time.</p>";
              updateTimestamp("breakoutUpdated");
              return;
            }
            let html = "<table><thead><tr><th>Ticker</th><th>Latest Price</th><th>20-Day High</th></tr></thead><tbody>";
            breakouts.forEach(item => {
              html += `<tr>
                        <td>${item.ticker}</td>
                        <td>${item.latestPrice.toFixed(2)}</td>
                        <td>${item.recentHigh.toFixed(2)}</td>
                      </tr>`;
            });
            html += "</tbody></table>";
            container.innerHTML = html;
            updateTimestamp("breakoutUpdated");
          })
          .catch(err => { container.innerHTML = `<p>Error updating breakout alerts: ${err}</p>`; });
      }
      
      // Volatility Monitor
      function fetchVolatilityData(ticker) {
        const apiKey = "0M17XRJD0VADHB6I";
        const url = `https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=${ticker}&apikey=${apiKey}`;
        return fetch(url)
          .then(response => response.json())
          .then(data => {
            const ts = data["Time Series (Daily)"];
            if (!ts) return null;
            const dates = Object.keys(ts).sort();
            const latestDay = dates[dates.length - 1];
            const high = parseFloat(ts[latestDay]["2. high"]);
            const low = parseFloat(ts[latestDay]["3. low"]);
            const volatility = ((high - low) / low) * 100;
            return { ticker, high, low, volatility };
          })
          .catch(err => { console.error("Error fetching volatility for", ticker, err); return null; });
      }
      function updateVolatilityMonitor() {
        const container = document.getElementById("volatilityMonitor");
        container.innerHTML = "<div class='spinner'></div>";
        Promise.all(stockList.map(ticker => fetchVolatilityData(ticker)))
          .then(results => {
            const valid = results.filter(r => r !== null);
            if(valid.length === 0) { container.innerHTML = "<p>No volatility data available.</p>"; return; }
            let html = "<table><thead><tr><th>Ticker</th><th>High</th><th>Low</th><th>Volatility (%)</th></tr></thead><tbody>";
            valid.forEach(item => {
              html += `<tr>
                        <td>${item.ticker}</td>
                        <td>${item.high.toFixed(2)}</td>
                        <td>${item.low.toFixed(2)}</td>
                        <td>${item.volatility.toFixed(2)}</td>
                      </tr>`;
            });
            html += "</tbody></table>";
            container.innerHTML = html;
            updateTimestamp("volatilityUpdated");
          })
          .catch(err => { container.innerHTML = `<p>Error updating volatility: ${err}</p>`; });
      }
      
      // Moving Average Crossover Alerts
      function fetchCrossoverData(ticker) {
        const apiKey = "0M17XRJD0VADHB6I";
        const url = `https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=${ticker}&apikey=${apiKey}`;
        return fetch(url)
          .then(response => response.json())
          .then(data => {
            const ts = data["Time Series (Daily)"];
            if (!ts) return null;
            const dates = Object.keys(ts).sort();
            if (dates.length < 200) return null;
            const last50 = dates.slice(-50).map(date => parseFloat(ts[date]["4. close"]));
            const last200 = dates.slice(-200).map(date => parseFloat(ts[date]["4. close"]));
            const ma50 = last50.reduce((a, b) => a + b, 0) / last50.length;
            const ma200 = last200.reduce((a, b) => a + b, 0) / last200.length;
            const signal = ma50 > ma200 ? "Golden Cross" : "Death Cross";
            return { ticker, ma50, ma200, signal };
          })
          .catch(err => { console.error("Error fetching crossover data for", ticker, err); return null; });
      }
      function updateCrossoverAlerts() {
        const container = document.getElementById("crossoverAlerts");
        container.innerHTML = "<div class='spinner'></div>";
        Promise.all(stockList.map(ticker => fetchCrossoverData(ticker)))
          .then(results => {
            const valid = results.filter(r => r !== null);
            if (valid.length === 0) {
              container.innerHTML = "<p>No crossover alerts available.</p>";
              updateTimestamp("crossoverUpdated");
              return;
            }
            let html = "<table><thead><tr><th>Ticker</th><th>50-Day MA</th><th>200-Day MA</th><th>Signal</th></tr></thead><tbody>";
            valid.forEach(item => {
               html += `<tr>
                         <td>${item.ticker}</td>
                         <td>${item.ma50.toFixed(2)}</td>
                         <td>${item.ma200.toFixed(2)}</td>
                         <td>${item.signal}</td>
                       </tr>`;
            });
            html += "</tbody></table>";
            container.innerHTML = html;
            updateTimestamp("crossoverUpdated");
          })
          .catch(err => { container.innerHTML = `<p>Error updating crossover alerts: ${err}</p>`; });
      }
      
      // Price Alert Monitor
      function checkPriceAlert() {
        const ticker = document.getElementById("alertTicker").value.trim();
        const alertPrice = parseFloat(document.getElementById("alertPrice").value);
        if (!ticker || isNaN(alertPrice)) {
          alert("Please enter both a valid ticker and alert price.");
          return;
        }
        const apiKey = "0M17XRJD0VADHB6I";
        const url = `https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=${ticker}&apikey=${apiKey}`;
        fetch(url)
          .then(response => response.json())
          .then(data => {
            const ts = data["Time Series (Daily)"];
            if (!ts) { document.getElementById("priceAlertResult").innerHTML = "No data available."; return; }
            const dates = Object.keys(ts).sort();
            const latestPrice = parseFloat(ts[dates[dates.length - 1]]["4. close"]);
            let message = `Current Price of ${ticker}: $${latestPrice.toFixed(2)}. `;
            if (latestPrice >= alertPrice) {
              message += `<strong>Alert: Price has reached/exceeded your target!</strong>`;
            } else {
              message += `Price is below your target.`;
            }
            document.getElementById("priceAlertResult").innerHTML = message;
          })
          .catch(err => { document.getElementById("priceAlertResult").innerHTML = `Error: ${err}`; });
      }
      
      // Initial Load & Auto-Update Intervals
      updateGroupTicker(1);
      updateGroupTicker(2);
      updateGroupTicker(3);
      updateGroupTicker(4);
      updateShortSqueezeLeaderboard();
      updateVolumeSpikeLeaderboard();
      updateNewsSentiment();
      updateBreakoutAlerts();
      updateVolatilityMonitor();
      updateCrossoverAlerts();
      
      setInterval(updateShortSqueezeLeaderboard, 60000);
      setInterval(updateVolumeSpikeLeaderboard, 60000);
      setInterval(updateNewsSentiment, 120000);
      setInterval(updateBreakoutAlerts, 60000);
      setInterval(updateVolatilityMonitor, 60000);
      setInterval(updateCrossoverAlerts, 60000);
    });
  </script>
</body>
</html>
"""))

In [10]:
# soon to incorporate this charting method: https://github.com/twopirllc/pandas-ta/blob/main/examples/example.ipynb

In [11]:
# en fin

# Disclaimer

All investments involve risk, and the past performance of a security, industry, sector, market, financial product, trading strategy, or individual’s trading does not guarantee future results or returns. Investors are fully responsible for any investment decisions they make. Such decisions should be based solely on an evaluation of their financial circumstances, investment objectives, risk tolerance, and liquidity needs.

Any opinions, news, research, analyses, prices, or other information offered is provided as general market commentary, and does not constitute investment advice. We will not accept liability for any loss or damage, including without limitation any loss of profit, which may arise directly or indirectly from use of or reliance on such information.


In [12]:
# 3eN fiN